In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from pytexit import py2tex
from IPython.display import Markdown
from gekko import GEKKO
import pandas as pd
from matplotlib import rcParams
import os
import sys
from copy import copy


%matplotlib widget
#%matplotlib inline

# Look for custom python codes in the Radigan Engineering Vault...
sys.path.insert(1, 'C:\\\\RadiganVault\\\\Python')

import RadiganEngineeringDefaults as RE
from RadiganEngineering_Materials_Library import *
from RadiganEngineeringTorsionSpring import *
from RadiganEngineeringBellevilleSpring import *

# Use higher quality figures in PDF exports
#rcParams['figure.dpi']= 300
#rcParams['figure.figsize'] = [8, 6]
set_matplotlib_formats('png', 'pdf')

print(ALUMINUM_6061_T6.Specific_Heat)

896 joule / kelvin / kilogram


In [19]:
# https://apmonitor.com/pdc/index.php/Main/TCLabSim
# https://apmonitor.com/pdc/index.php/Main/TCLabConvection

# Update model with natural convection estimation.

TCLab = GEKKO()    # create GEKKO model for dx/dt

t_sim = 360 * ureg.sec # Number of seconds to simulate
points_per_second = 1 # Only works if this is the same as the 'sampling frequency' in the cell below...

TCLab.time = np.linspace(0, t_sim.magnitude, (t_sim.magnitude * points_per_second) + 1) * ureg.sec # time points

# dt is in the units of ________.

T_Ambient = 26.0 # degC
HS_Mass  = 0.004 * ureg.kg 
HS_Cp    = 500.0 * (ureg.joule /  (ureg.kg    * ureg.kelvin)) # Heatsink Specific Heat
HS_Conv  = 10.0  * (ureg.watt) / ((ureg.m**2) * ureg.kelvin)  # Convection coefficient
HS_Area  = 0.0012 * ureg.m**2                                 # Heatsink Surface Area

# Starting temperature for the heatsink.
# Might be equal to 'T_Ambient' or something else.
#HS_T_Start = T_Ambient
HS_T_Start = 28.0


HeaterValues = np.zeros(len(TCLab.time))
HeaterValues[15:75] = 50
#HeaterValues[0:9] = 50
Heater_Q  = TCLab.Param(HeaterValues)  # "Percentage of maximum value" of the heater -- set the heater percentage down below 

K_p       = TCLab.Param()  # ~= 0.01 Watts / % # This seems a little low, I would have expected more than 1 Watt at full power [100%]
HS_Temp     = TCLab.Var(HS_T_Start) # GEKKO variable for the temperature of the heatsink

TCLab.Equation(HS_Temp.dt()  == ( (HS_Conv.to('watt/(m^2*K)').magnitude * HS_Area.to('m^2').magnitude) * (T_Ambient - HS_Temp) + (K_p * Heater_Q) ) / 
                                    (HS_Mass.to('kg').magnitude * HS_Cp.to('joule/(kg*K)').magnitude) )

# solve ODEs and plot
# https://gekko.readthedocs.io/en/latest/imode.html
TCLab.options.IMODE = 4 # dynamic simulation

# Type: Integer, Input/Output
# Default Value: 0
# Description: Cold start model: 0=warm start, 1=cold start, 2=decompose problem
#TCLab.options.COLDSTART = 2

#TCLab.options.SOLVER = 1
#TCLab.options.MAX_ITER = 1000

#---------------------
TCLab.options.NODES = 6
#---------------------

TCLab.options.TIME_SHIFT=0


fig = plt.figure()
plt_ax = plt.axes(label='TCLab')
plt_ax.set_ylim(25,45)

#HeaterValue = 50
#Heater_Q.value = HeaterValue # Turn the heater on to the specified percentage value
#print(Heater_Q.value)

#k_vals = [10, 20, 30, 50]
# Solve the dynamic problem for this set of Kp values.
K_p_vals = [0.008, 0.01]

for K_p_val in K_p_vals:
    K_p.value = K_p_val
    TCLab.solve(disp=False)
    plt_ax.plot(TCLab.time,HS_Temp, marker='',linewidth=1,label='K_p = {0}'.format(K_p.value[0]))


######################################################################################
# Create the 'dummy' plot with NaN's that will be replaced with measured data below.
T1 = np.empty(len(TCLab.time))
T1[:] = np.nan
T1_time = np.empty(len(TCLab.time))
T1_time[:] = np.nan
MeasuredData, = plt_ax.plot(T1_time,T1,'r.', label='Measured')
######################################################################################

plt.grid(True)
plt.title('TC Lab Thermal Simulation')
plt.xlabel('Time (sec)')
plt.ylabel('Temperature (degC)')
plt.legend()
plt.show()

# instantiate a second axes that shares the same x-axis and plot the Heater Input
Q_ax = plt_ax.twinx()  
Q_ax.plot(TCLab.time,Heater_Q, color = 'gray', linestyle = ':', marker='',linewidth=2,label='Heater')
Q_ax.set_ylim(-5,105)
plt.ylabel('Heater Percentage')
plt.legend(loc = 'lower right')

# Now solve as a steady-state problem in case the simulation time isn't long enough.
# Sometimes the parameter values doen't seem to 'stick'...
#TCLab.options.IMODE = 3
#TCLab.solve(disp=False)
#print('Final Temperature = {0} for a Heater Value of {1}% and a K_p of {2}'.format(HS_Temp.value[0], HeaterValue, K_p.value[0]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\ProgramData\Anaconda3\lib\site-packages\gekko\gk_write_files.py:144: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  csv_data = np.hstack(('time',np.array(self.time).flatten().astype(object)))
C:\ProgramData\Anaconda3\lib\site-packages\gekko\gk_write_files.py:144: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  csv_data = np.hstack(('time',np.array(self.time).flatten().astype(object)))


In [20]:
# Actual measured data
import tclab
import time

if (1):
    lab = tclab.TCLab()
    #T1 = [lab.T1] # Old-style list based plotting...
    
    # Static On / Off Heater control
    #lab.Q1(50)
    #print('Turning Heater on to {0}%'.format(HeaterValue))
    #lab.Q1(HeaterValue)
    
    # Use the array defined above to set the heater.
    lab.Q1(HeaterValues[0])

    T1[:] = np.nan       # Reset the measured temperatures to NaN's
    T1_time[:] = np.nan

    for i in range(len(TCLab.time)):
        time.sleep(1)
        lab.Q1(HeaterValues[i])
        #print(lab.T1)
        print('Time = {3} sec, T1 = {0}, T2 = {1}, Heater Set to {2}'.format(lab.T1, lab.T2, HeaterValues[i], i))
        T1[i] = lab.T1
        T1_time[i] = i
        MeasuredData.set_ydata(np.array(T1))
        MeasuredData.set_xdata(np.array(T1_time))
        plt_ax.set_title("{0} of {1} Points Measured".format(i+1,t_sim))
        fig.canvas.draw()
        fig.canvas.flush_events()
        plt.show()

    lab.close()
    plt.show()


TCLab version 0.4.9
Arduino Leonardo connected on port COM3 at 115200 baud.
TCLab Firmware 1.4.3 Arduino Leonardo/Micro.
Time = 0 sec, T1 = 28.0, T2 = 27.67, Heater Set to 0.0
Time = 1 sec, T1 = 28.0, T2 = 27.67, Heater Set to 0.0
Time = 2 sec, T1 = 28.0, T2 = 27.67, Heater Set to 0.0
Time = 3 sec, T1 = 28.0, T2 = 27.67, Heater Set to 0.0
Time = 4 sec, T1 = 28.0, T2 = 27.67, Heater Set to 0.0
Time = 5 sec, T1 = 28.0, T2 = 27.67, Heater Set to 0.0
Time = 6 sec, T1 = 28.0, T2 = 27.67, Heater Set to 0.0
Time = 7 sec, T1 = 28.0, T2 = 27.67, Heater Set to 0.0
Time = 8 sec, T1 = 28.0, T2 = 27.67, Heater Set to 0.0
Time = 9 sec, T1 = 28.0, T2 = 27.67, Heater Set to 0.0
Time = 10 sec, T1 = 28.0, T2 = 27.67, Heater Set to 0.0
Time = 11 sec, T1 = 28.0, T2 = 27.67, Heater Set to 0.0
Time = 12 sec, T1 = 28.0, T2 = 27.67, Heater Set to 0.0
Time = 13 sec, T1 = 28.0, T2 = 27.67, Heater Set to 0.0
Time = 14 sec, T1 = 28.0, T2 = 27.67, Heater Set to 0.0
Time = 15 sec, T1 = 28.0, T2 = 27.67, Heater Set 

In [18]:
if (1): # Grab the current temperature from the TCLab if desired. (I.e. = 1)
    import tclab
    lab = tclab.TCLab()
    print('T1 = {0}, T2 = {1}'.format(lab.T1, lab.T2))
    print('T1 = {0}, T2 = {1}'.format(lab.T1, lab.T2))
    print('T1 = {0}, T2 = {1}'.format(lab.T1, lab.T2))
    lab.close()

TCLab version 0.4.9
Arduino Leonardo connected on port COM3 at 115200 baud.
TCLab Firmware 1.4.3 Arduino Leonardo/Micro.
T1 = 28.0, T2 = 27.67
T1 = 28.0, T2 = 27.67
T1 = 28.0, T2 = 27.67
TCLab disconnected successfully.


In [7]:
#lab.close()

TCLab disconnected successfully.
